In [3]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import time
import sklearn
from tensorflow.keras.models import load_model, Sequential
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, GRU, Dense, Dropout, BatchNormalization, LSTM, Lambda
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import KernelPCA
from random import shuffle

from model_creation import create_cnn_lstm_model, create_cnn_gru_model


In [4]:
data_full = pd.read_csv("data_full.csv", header=0)
data_full['DATE (MM/DD/YYYY)'] = pd.to_datetime(data_full['DATE (MM/DD/YYYY)'], format='%Y-%m-%d %H:%M:%S')
data_full.set_index('DATE (MM/DD/YYYY)', inplace=True)

# Resample data to 1-hour intervals using mean
data_full = data_full.resample('1h').mean()
data_full.drop(columns=[
    'Global CMP22 (vent/cor) [W/m^2]',
    'Global CM6b (cor) [W/m^2]',
    'Global CM3 (cor) [W/m^2]',
    'Global PSP (vent/cor) [W/m^2]',
    'Global LI-200 [W/m^2]',
    'Global TSP-700 Vent [W/m^2]',
    'Global SP Lite2 [W/m^2]',
    'Global SP-110 [W/m^2]',
    # 'Tower Wet Bulb Temp [deg C]',
    # 'Total Cloud Cover [%]',
    # 'Direct Extraterrestrial (calc) [W/m^2]',
    # 'Tower Wind Chill Temp [deg C]'
], inplace=True)

print("Original data shape:", data_full.shape)
print("Hourly data shape:", data_full.shape)
data_full.head()

def bad_correlation_with_output(dataset, threshold):
   col_corr = set()  # Set of all the names of correlated columns
   corr_matrix = dataset.corr(method='spearman')  # Calculate Spearman correlation
   num_columns = len(corr_matrix.columns) 
   for i in range(num_columns):
      if i != 0:
         if abs(corr_matrix.iloc[i,17 ]) < threshold:
               colname = corr_matrix.columns[i]
               col_corr.add(colname)
   return col_corr

bad_features = bad_correlation_with_output(data_full, 0.2)
data_full.drop(columns=bad_features, inplace=True)

scaler = MinMaxScaler(feature_range=(0, 1))
data = scaler.fit_transform(data_full.to_numpy())
data = pd.DataFrame(data, columns=list(data_full.columns))

data


Original data shape: (96408, 18)
Hourly data shape: (96408, 18)


,Zenith Angle [degrees],Airmass,Tower Dry Bulb Temp [deg C],Tower Wet Bulb Temp [deg C],Tower Wind Chill Temp [deg C],Tower RH [%],Total Cloud Cover [%],Opaque Cloud Cover [%],Avg Wind Direction @ 19ft [deg from N],Global Extraterrestrial (calc) [W/m^2],Direct Extraterrestrial (calc) [W/m^2],GHI Mean
0,0.998386,0.0,0.243027,0.329259,0.295364,0.668347,0.0,0.0,0.818025,0.0,0.0,0.0
1,0.951385,0.0,0.247419,0.329655,0.267839,0.609294,0.0,0.0,0.799517,0.0,0.0,0.0
2,0.880887,0.0,0.257840,0.337691,0.247046,0.551588,0.0,0.0,0.843259,0.0,0.0,0.0
3,0.803078,0.0,0.255738,0.340131,0.299806,0.603448,0.0,0.0,0.854910,0.0,0.0,0.0
4,0.723579,0.0,0.250776,0.336817,0.288904,0.637339,0.0,0.0,0.495270,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
96403,0.711739,0.0,0.401330,0.509611,0.443696,0.486831,0.0,0.0,0.092505,0.0,0.0,0.0
96404,0.791210,0.0,0.390837,0.501292,0.434802,0.515518,0.0,0.0,0.418076,0.0,0.0,0.0
96405,0.869515,0.0,0.387838,0.499476,0.432020,0.525806,0.0,0.0,0.839479,0.0,0.0,0.0
96406,0.941812,0.0,0.396782,0.507094,0.440318,0.505057,0.0,0.0,0.855859,0.0,0.0,0.0


In [5]:
def singleStepSampler(df, window):
    # Convert DataFrame to numpy array for faster operations
    data = df.values
    n = len(data)

    # Create sliding windows using array indexing
    x = np.array([data[i:i+window] for i in range(n - window)])

    # Get the target values (GHI Mean is the first column)
    y = data[window:, 11]  # Assuming 'GHI Mean' is the first column

    return x, y.reshape(-1, 1)  # Reshape y to be 2D (n_samples, 1)

(xVal, yVal) = singleStepSampler(data, 24)

SPLIT = 0.8

train_threshold = int(SPLIT * len(xVal))
val_threshold = int((1 - (1 - SPLIT) / 2) * len(xVal))

X_train = xVal[:train_threshold]
Y_train = yVal[:train_threshold]
X_val = xVal[train_threshold:val_threshold]
Y_val = yVal[train_threshold:val_threshold]
X_test = xVal[val_threshold:]
Y_test = yVal[val_threshold:]

# Output the size of the dataset
print("X_train shape:", X_train.shape)
print("Y_train shape:", Y_train.shape)
print("X_val shape:", X_val.shape)
print("Y_val shape:", Y_val.shape)
print("X_test shape:", X_test.shape)
print("Y_test shape:", Y_test.shape)


X_train shape: (77107, 24, 12)
Y_train shape: (77107, 1)
X_val shape: (9638, 24, 12)
Y_val shape: (9638, 1)
X_test shape: (9639, 24, 12)
Y_test shape: (9639, 1)


In [ ]:
def custom_activation(x):
    x1 = x[:, 0:1]
    x2 = x[:, 1:2]
    x2 = tf.keras.activations.sigmoid(x2)  # Apply sigmoid only to second output
    return tf.concat([x1, x2], axis=1)

actor = Sequential([
   Input(shape=(1,)),
   Dense(32, activation="relu"),
   Dense(2),
   Lambda(custom_activation)
])

critic = Sequential([
   Input(shape=(1,)),
   Dense(32, activation="relu"),
   Dense(1)
])

actor_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
critic_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

In [ ]:
num_episodes = 1

for episode in range(num_episodes):
   num_models = 2
   all_losses = np.zeros(num_models)
   with tf.GradientTape(persistent=True) as tape:
      print(f"Episode {episode + 1}")

      for i in range(num_models):
         actor_prediction = actor(np.array([1]))
         mean, sigma = actor_prediction[0][0], actor_prediction[0][1]
         adam_gradient = np.exp(np.random.normal(mean, sigma))

         # Create the model with the predicted learning rate
         input_shape = (X_train.shape[1], X_train.shape[2])
         model = create_cnn_gru_model(input_shape)
         optimizer = Adam(learning_rate=adam_gradient)
         model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

         model.fit(
            X_train, Y_train,
            validation_data=(X_val, Y_val),
            batch_size=32,
            epochs=1,
            verbose=1
         )
         
         # Test the model
         test_loss, _ = model.evaluate(X_test, Y_test, verbose=1)
         print(f"Adam Gradient: {adam_gradient:.4f}")
         print(f"Test Loss: {test_loss:.4f}")

         all_losses[i] = test_loss

      mean_loss = np.mean(all_losses)
      best_loss = np.min(all_losses)
      reward = -np.square(mean_loss - best_loss)
      
      state_value = critic(np.array([mean_loss]))

Episode 1
2410/2410 ━━━━━━━━━━━━━━━━━━━━ 46s 10ms/step - loss: 1.8931 - mae: 0.9187 - val_loss: 1.3945 - val_mae: 1.1561
302/302 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 1.3786 - mae: 1.1504
Adam Gradient: 0.9145
Test Loss: 1.3786
2410/2410 ━━━━━━━━━━━━━━━━━━━━ 43s 11ms/step - loss: 1.8951 - mae: 0.8367 - val_loss: 0.5347 - val_mae: 0.6910
302/302 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.5401 - mae: 0.6965
Adam Gradient: 0.9184
Test Loss: 0.5401
[1.37858307 0.54005295]
